In [ ]:
#%%appyter init
from appyter import magic
magic.init(lambda _=globals: _())

In [ ]:
import json
import requests
from time import sleep
from tabulate import tabulate
from IPython.display import HTML, display

In [ ]:
def get_kea3_results(gene_set, query_name):
    ADDLIST_URL = 'https://amp.pharm.mssm.edu/kea3/api/enrich/'
    payload = {
        'gene_set': gene_set,
        'query_name': query_name
    }

    response = requests.post(ADDLIST_URL, data=json.dumps(payload))
    if not response.ok:
        raise Exception('Error analyzing gene list')
    sleep(1)

    return json.loads(response.text)

In [ ]:
%%appyter hide_code

{% do SectionField(name ='section', title = 'Perform Kinase Enrichment Analysis (KEA) 3', 
                   subtitle = 'Kinase Enrichment Analysis 3 (KEA3) infers upstream kinases whose putative substrates are ' +
                               'overrepresented in a user-inputted list of genes or differentially phosphorylated proteins. ' +
                               'The KEA3 database contains putative kinase-substrate interactions collected from publicly ' +
                               'available datasets. Upload -OR- input a gene/protein list to retrieve the top associated kinases.', 
                   img = 'KEA3_transparent_logo.png')%}

In [ ]:
%%appyter markdown

### Generate table containing top associated kinases for different libraries
Libraries are supersets of kinase substrate sets that are aggregated based on the database from which they are derived.

In [ ]:
%%appyter code_exec

gene_list_file = {{ FileField(
        name = 'Upload gene/protein list',
        label = 'Upload gene/protein list',
        default = '',
        section = 'section'
    )}}

gene_list_input = {{ TextField(
        name = 'Input gene/protein list',
        label = 'Input gene/protein list (one per row)',
        default = 'ZNF264 \nTMPO \nISL2 \nMAP3K8 \nEFNB1 \nEIF3C \nOSBPL11 \nABCF1 \nUTRN \nOPRK1 \nTSC1 ' +
                    '\nGAB2 \nRPS3P2 \nDDX3X \nPPP1CA \nNF2 \nRBM3 \nIRAK1 \nKCNH2 \nNPR1 \nMOCOS \nITSN2 ' +
                    '\nMITF \nARAF \nDAPK2 \nEPHB2 \nCACNA1G \nYWHAZ \nGMFB',
        section = 'section',
    )}}

# Number of ranked kinases to display
num_kinases = {{ IntField(
        name = 'Input number of top kinases to display',
        label = 'Number of top kinases to display',
        description = 'Input any integer from 1 to 100',
        default = 10,
        min = 1, 
        max = 100,
        section = 'section'
    )}}

input_exists = False 

if gene_list_file != '':
    open_gene_list_file = open(gene_list_file,'r')
    lines = open_gene_list_file.readlines()
    genes = [x.strip() for x in lines]
    open_gene_list_file.close()
    input_exists = True 
elif gene_list_input != '':
    genes = gene_list_input.split('\n')
    genes = [x.strip() for x in genes]
    input_exists = True 
else:
    display(HTML('<h2>No kinase list was inputted or uploaded.</h2>'))

if input_exists:
    results = get_kea3_results(genes, 'Query')

    # Create dictionary of dictionaries for the top 10 kinases for each ranking type
    # Format: rankings['Rank Type'] = {1: Kinase, 2: Kinase, 3: Kinase, ...}
    rankings = {}
    for library in range(0, 13):
        rankings[list(results.keys())[library]] = {}
        for rank in range(1, num_kinases+1):
            rankings[list(results.keys())[library]].update({rank: list(results.values())[library][rank-1]['TF']})

    # Create list of lists
    # Format: ranks[0] will contain all of the #1 kinases for each ranking type 
    ranks = []
    for rank_num in range(0, num_kinases+1):
        ranks.append([rank_num+1])

    for library in range(0, 13):
        for rank in range(1, num_kinases+1):
            ranks[rank-1].append(list(rankings[list(results.keys())[library]].values())[rank-1])

    list_all_rankings = []
    for num in range(0, num_kinases):
        list_all_rankings.append(ranks[num])
    
    list_all_headers = []
    for library in range(0, 13):
        list_all_headers.append(list(rankings.keys())[library])
            
    display(HTML(tabulate(list_all_rankings, headers = list_all_headers, tablefmt = 'html' )))
